In [ ]:
#Scrapple
##Takes HL staff directory URL and returns CSV for use in Virtualitics
##Matt Cook - 2022
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd
import time
import os
import sys

#declarations
links = []
locations = [] 
expanded = False
table = "...OrgViz - input.csv"
out = "...OrgViz - output.csv"
staff = {}
count = 0

#initialize automated browser and expand directory page
driver = webdriver.Safari()
driver.get("https://library.harvard.edu/staff?")
            
#get individual staff page links from directory page
def get_Links():    
    elems = driver.find_elements(By.XPATH, "//a[@href]")
    for elem in elems:
        link = (elem.get_attribute("href"))
        if str("staff/") in link:
            links.append(link)
    
#get location data from individual staff pages    
def get_NameLoc():
    for link in links:
        driver.get(link)
        name = driver.find_element(By.CLASS_NAME, "hl__page-title")
        name = str(name.text)
        staff[name] = [link]
        try:
            location = driver.find_element(By.CLASS_NAME, "hl__contact-info__address")
            location = location.text.splitlines()[1]
            location = location.split(',')
            location = location[0]
            location = location.lstrip()
            staff[name] += [location]
            print((str(name) + " works at " + (str(location))))
        except NoSuchElementException:
            ("no location found")

#Locate and click "load more" staff profile button until staff list fully expanded
while expanded == False:
    try:
        #loadMore = driver.find_element(By.XPATH, '//*[@id="block-harvard-content"]/article/div/div/div/div[2]/section/div/button')
        loadMore = driver.find_element(By.CLASS_NAME, "hl__filter-bar-directory__load-more")
        time.sleep(3)
        print("Expanding staff list on library website...")
        loadMore.click()
    except StaleElementReferenceException:
        print("\n")
        print("Staff list fully expanded. Identifying locations now...")
        expanded == True
        get_Links()
        get_NameLoc()
        break
    except NoSuchElementException:
        print("\n")
        print("Staff list fully expanded. Identifying locations now...")
        expanded == True
        get_Links()
        get_NameLoc()
        break
print("\n")
print("Staff dictionary successfully compiled. Outputting locations to csv...")

#append input Csv with matching location values
fileCore = pd.read_csv(table)
while count < len(fileCore.index):
    lastName = fileCore.iloc[count]["Employee Last Name"]
    for k,v in staff.items():
        if len(v) == 2:
            k = k.split()
            k = k[1]
            if str(lastName) in k:
                fileCore.loc[count, 'Location']= v[1]
                #print(lastName + " location = " + (str(v[1])))
    count = count + 1               
print("\n")

#output new csv from new dataframe
with open(out, mode = 'w') as f:
    fileCore.to_csv(f)
    print("New CSV output to: " + out)
    
print("have a nice day")